# w08-3：JSON資料型態

### JSON資料

- https://zh.wikipedia.org/zh-tw/JSON

```
{"time":"2020-07-01", "data": [1, 2, 3, 4, 5]}
```

### 空氣品質應用
- 資料來源：https://data.gov.tw/dataset/40448


### JSON格式初探
- https://zh.wikipedia.org/zh-tw/JSON

In [1]:
import requests

In [2]:
#url = 'http://opendata.epa.gov.tw/ws/Data/AQI/?$format=json' # 已失效
url = 'https://data.epa.gov.tw/api/v2/aqx_p_432?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate%20desc&format=JSON'

In [3]:
r = requests.get(url)

In [4]:
r

<Response [200]>

In [5]:
data = r.json()

In [7]:
data['records']

[{'sitename': '基隆',
  'county': '基隆市',
  'aqi': '32',
  'pollutant': '',
  'status': '良好',
  'so2': '1.5',
  'co': '0.21',
  'o3': '32.3',
  'o3_8hr': '35.3',
  'pm10': '19',
  'pm2.5': '6',
  'no2': '4.4',
  'nox': '5.3',
  'no': '0.9',
  'wind_speed': '2.5',
  'wind_direc': '97',
  'publishtime': '2022/12/17 11:00:00',
  'co_8hr': '0.2',
  'pm2.5_avg': '5',
  'pm10_avg': '13',
  'so2_avg': '1',
  'longitude': '121.760056',
  'latitude': '25.129167',
  'siteid': '1'},
 {'sitename': '汐止',
  'county': '新北市',
  'aqi': '31',
  'pollutant': '',
  'status': '良好',
  'so2': '1.1',
  'co': '0.22',
  'o3': '27.3',
  'o3_8hr': '33',
  'pm10': '3',
  'pm2.5': '3',
  'no2': '7.5',
  'nox': '9.5',
  'no': '1.9',
  'wind_speed': '1.4',
  'wind_direc': '15',
  'publishtime': '2022/12/17 11:00:00',
  'co_8hr': '0.2',
  'pm2.5_avg': '2',
  'pm10_avg': '3',
  'so2_avg': '0',
  'longitude': '121.6423',
  'latitude': '25.067131',
  'siteid': '2'},
 {'sitename': '萬里',
  'county': '新北市',
  'aqi': '39',
  'p

In [8]:
records = data['records']

In [12]:
for item in records:
    if item['county']=='臺南市':
        print(item['county'], item['sitename'], item['pm2.5'])

臺南市 新營 2
臺南市 善化 2
臺南市 安南 6
臺南市 臺南 11
臺南市 臺南(麻豆) 5
臺南市 臺南(學甲) 10


- 小試身手: 把高雄站點的AQI, 硫化物, 還有GPS座標印出來

In [13]:
# 把程式填入這邊
for item in records:
    if item['county']=='臺南市':
        print(item['county'], item['sitename'], item['pm2.5'], item['latitude'], item['longitude'])


臺南市 新營 2 23.305633 120.31725
臺南市 善化 2 23.11337642 120.29740529
臺南市 安南 6 23.048197 120.2175
臺南市 臺南 11 22.98525833 120.22238056
臺南市 臺南(麻豆) 5 23.17904722 120.24583056
臺南市 臺南(學甲) 10 23.23688056 120.18339722


### 進一步找出 全台有超標的站，並列出來
- AQI 是否超標，請參照以下的資訊
- ref: https://airtw.epa.gov.tw/CHT/Information/Standard/AirQualityIndicator.aspx

In [19]:
for item in records:
    aqi = float(records[0]['aqi'])


32.0

In [22]:
# 把程式填入這邊
for item in records:
    aqi = float(item['aqi'])
    if aqi>50:
        print(item['county'], item['sitename'], item['aqi'], item['latitude'], item['longitude'])


新北市 富貴角 56 25.29681695 121.53656894
屏東縣 屏東(琉球) 56 22.35222 120.37722
屏東縣 屏東(枋山) 60 22.260899 120.651472


### 在地圖上把所有的站畫出來

In [28]:
# 把程式填入這邊

import folium
import geocoder

# 使用 geocoder 取得特定住址的 GPS 座標
location = geocoder.osm('台灣').latlng
print(location)


m = folium.Map(location=location, zoom_start=12)

for item in records:
    try:
        name = item['sitename']  #站名
        lat = item['latitude']  #緯度
        lng = item['longitude']  #經度
        aqi = int(item['aqi'])
        info = '[%s] %s' %(name, aqi)
        folium.Marker([ float(lat), float(lng)],
                      tooltip=info,
                     icon=folium.Icon(color='blue')
                     ).add_to(m)
        
    except Exception as e:
        print(e.args)    

[23.9739374, 120.9820179]


In [29]:
m

### 畫出地圖，並且把超標的畫紅色、沒超標的畫綠色

In [37]:
# 把程式填入這邊

import folium
import geocoder

url = 'https://data.epa.gov.tw/api/v2/aqx_p_432?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate%20desc&format=JSON'
r = requests.get(url)
print(r)
data = r.json()
records = data['records']
print(len(records))

# 使用 geocoder 取得特定住址的 GPS 座標
location = geocoder.osm('台灣').latlng
print(location)

m = folium.Map(location=location, zoom_start=10)

for item in records:
    try:
        name = item['sitename']  #站名
        lat = item['latitude']  #緯度
        lng = item['longitude']  #經度
        aqi = int(item['aqi'])
        info = '[%s] %s' %(name, aqi)
        if aqi>50:
            folium.Marker([ float(lat), float(lng)],
                          tooltip=info,
                         icon=folium.Icon(color='red')
                         ).add_to(m)
        else:
            folium.Marker([ float(lat), float(lng)],
                          tooltip=info,
                         icon=folium.Icon(color='green')
                         ).add_to(m)
            
        
    except Exception as e:
        print(e.args)    

<Response [200]>
0
[23.9739374, 120.9820179]


In [35]:
m

### 把成果下載下來欣賞

In [ ]:
# 寫下你的程式
m.save('TW-air.html')